In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
# 读取数据
data = pd.read_csv("原始数据/ruc_Class25Q1_test.csv")

In [3]:
# 清洗数据
## 户型
def parse_room_layout(layout):
    result = {
        '室': 0,
        '厅': 0,
        '厨': 0,
        '卫': 0
    }
    # 对于空值，返回默认值
    if isinstance(layout, float):
        return pd.Series({'室': 0, '厅': 0, '厨': 0, '卫': 0})
    # 处理"房间"和"室"的别名
    layout = layout.replace('房间', '室')
    
    # 使用正则表达式查找所有数字+单位组合
    matches = re.findall(r'(\d+)(室|厅|厨|卫)', layout)
    
    for num, unit in matches:
        result[unit] = int(num)
    
    return pd.Series(result)

data[['室', '厅', '厨', '卫']] = data['房屋户型'].apply(parse_room_layout)

In [4]:
## 所在楼层
def parse_floor_layout(floor_str):
    # 初始化默认值
    result = {
        '楼层位置': None,
        '楼层分组': None
    }
    
    # 处理缺失值
    if pd.isna(floor_str):
        return pd.Series(result)
    
    # 提取基本信息
    match = re.search(r'([底低中高顶](?:楼)?层)[^\d]*\(共(\d+)层\)', str(floor_str))
    if not match:
        return pd.Series(result)
    
    position, total_floors = match.groups()
    
    # 标准化楼层位置
    position = position[0]  # 取第一个字
    if position == '顶':
        position = '高'
    
    # 确定楼层分组
    try:
        total_floors = int(total_floors)
        if total_floors <= 6:
            group = '低层'
        elif 7 <= total_floors <= 15:
            group = '中层'
        else:
            group = '高层'
    except (ValueError, TypeError):
        group = None
    
    # 更新结果
    result.update({
        '楼层位置': position,
        '楼层分组': group
    })
    
    return pd.Series(result)
data[['层高','层型']] = data['所在楼层'].apply(parse_floor_layout)

In [5]:
## 房屋面积
def remove_units(series):
    # 复制Series以避免修改原数据
    result = series.copy()
    
    # 移除单位并转换为数值
    result = result.astype(str).str.replace(r'[^\d.]', '', regex=True)
    
    # 转换为float类型，非数字转为NaN
    return pd.to_numeric(result, errors='coerce')
    
data['建筑面积'] = remove_units(data['建筑面积'])
data['套内面积'] = remove_units(data['套内面积'])

In [6]:
## 交通出行
def extract_traffic(series):
    def get_type(text):
        if pd.isna(text):
            return None
        text = str(text)
        if '地铁' in text or '号线' in text:
            return '地铁'
        elif '公交' in text:
            return '公交'
    return series.apply(get_type)
data['交通出行'] = extract_traffic(data['交通出行'])

In [7]:
## 周边配套
def extract_surroundings(series):
    result = {
        '医院':0,
        '学校':0,
        '商场':0,
        '公园':0
    }

    if pd.isna(series):
        return pd.Series({'医院': 0, '学校': 0, '商场': 0,'公园':0})
        
    text = str(series)
    for facility in result:
        if facility in text:
            result[facility] = 1
    return pd.Series(result)
data[['医院','学校','商场','公园']] = data['周边配套'].apply(extract_surroundings)

In [8]:
## 梯户比
def extract_lift_to_house_ratio(text):
    if pd.isna(text):  # 处理 NaN
        return pd.Series({'梯户比': None})
    
    text = str(text)
    # 定义中文数字到阿拉伯数字的映射
    chinese_num_map = {
        '单': 1, '一': 1, '两': 2, '二': 2, '三': 3, '四': 4,
        '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '十': 10
    }

    pattern = r'([单两一二三四五六七八九十\d]+)梯([单两一二三四五六七八九十\d]+)户'
    match = re.search(pattern, text)
    
    if not match:
        return pd.Series({'梯户比': None})  # 未匹配到有效数据
    
    lift_str, house_str = match.groups()
    
    # 转换梯数（lift）
    if lift_str.isdigit():  # 如果是数字，直接转换
        lift = int(lift_str)
    else:  # 如果是中文数字，映射到阿拉伯数字
        lift = chinese_num_map.get(lift_str, 1)  # 默认 "单梯" = 1
    
    # 转换户数（house）
    if house_str.isdigit():  # 如果是数字，直接转换
        house = int(house_str)
    else:  # 如果是中文数字，解析（如 "二十五" = 25）
        # 处理 "二十五" → 2*10 + 5 = 25
        if '十' in house_str:
            parts = house_str.split('十')
            if len(parts) == 2:
                house = chinese_num_map.get(parts[0], 1) * 10 + chinese_num_map.get(parts[1], 0)
            else:  # "十" 单独出现，如 "十户" → 10
                house = 10
        else:
            house = chinese_num_map.get(house_str, 1)
    
    # 计算梯户比（梯数 / 户数）
    ratio = lift / house if house != 0 else None  # 避免除零错误
    return pd.Series({'梯户比': ratio})

data['梯户比'] = data['梯户比例'].apply(extract_lift_to_house_ratio)

In [9]:
## 交易时间差
def calculate_transaction_time_diff(df, current_time_col='交易时间', last_time_col='上次交易'):
    # 转换日期列为 datetime 格式
    df[current_time_col] = pd.to_datetime(df[current_time_col], errors='coerce')
    df[last_time_col] = pd.to_datetime(df[last_time_col], errors='coerce')
    
    # 计算时间差（天数）
    df['交易时间差（天）'] = (df[current_time_col] - df[last_time_col]).dt.days
    
    # 处理异常值（如 NaT 导致的 NaN）
    df['交易时间差（天）'] = df['交易时间差（天）'].fillna(-1)  # -1 表示无效数据
    
    return df

data = calculate_transaction_time_diff(data)

In [12]:
# 整合到一个csv文件中
selected_cols = ['ID','城市','区域','板块','环线','小区名称','建筑面积','套内面积','房屋朝向','建筑结构','装修情况','梯户比例','交易权属','上次交易','房屋用途','房屋年限','产权所属','lon','lat','年份','室','厅','厨','卫','层高','层型','医院','学校','商场','公园','梯户比','交易时间差（天）']
data[selected_cols].to_csv('test_result.csv', index=False, encoding='utf-8-sig')